<a href="https://colab.research.google.com/github/HamzaElshafie/PiCar_MLiS/blob/master/ResNet_Car/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/My Drive/MLiS_Car/training_data.zip" -d "/content/unzip"

In [ ]:
!cp "/content/drive/My Drive/MLiS_Car/training_norm.csv" "/content/"

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization, Layer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
import multiprocessing
from tensorflow.keras.layers import Layer
from tensorflow.keras.applications import ResNet50

In [ ]:
# Check GPU availability
print("GPU Available:", tf.test.is_gpu_available())

# Check TPU availability
tpu_available = False
devices = tf.config.list_logical_devices()
for device in devices:
    if device.device_type == 'TPU':
        tpu_available = True
        break

print("TPU Available:", tpu_available)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


GPU Available: True
TPU Available: False


In [ ]:
class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, targets, root_dir_original, batch_size=32, image_shape=(224, 224, 3), shuffle=True):
        self.targets = targets
        self.root_dir_original = root_dir_original
        self.batch_size = batch_size
        self.image_shape = image_shape
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return len(self.targets) // self.batch_size

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size: (index + 1) * self.batch_size]
        X, y = self.__data_generation(indexes)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.targets))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        X = np.empty((len(indexes), *self.image_shape))
        y = np.empty((len(indexes), 2))

        for i, index in enumerate(indexes):
            img_name = os.path.join(self.root_dir_original, str(int(self.targets.iloc[index]['image_id'])) + '.png')
            img = tf.keras.preprocessing.image.load_img(img_name, target_size=self.image_shape)
            img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
            X[i,] = img_array

            angle = self.targets.iloc[index]['angle']
            speed = self.targets.iloc[index]['speed']
            y[i,] = [angle, speed]

        return X, y

In [ ]:
# Load your data
targets_csv = '/content/training_norm.csv'  # Replace with the correct CSV file
root_dir_original = '/content/unzip/training_data/'

# Load targets
targets = pd.read_csv(targets_csv)

# Split the dataset
train_targets, test_val_targets = train_test_split(targets, test_size=0.3, random_state=42)
val_targets, test_targets = train_test_split(test_val_targets, test_size=0.5, random_state=42)

# Create data generators for training, validation, and test sets
train_dataset = CustomDataGenerator(targets=train_targets, root_dir_original=root_dir_original)
val_dataset = CustomDataGenerator(targets=val_targets, root_dir_original=root_dir_original)
test_dataset = CustomDataGenerator(targets=test_targets, root_dir_original=root_dir_original)

In [ ]:
# Load pre-trained ResNet50 model without top layers
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Dont Freeze the base model layers
base_model.trainable = True

# Define the model with ReLU activation for the output layer
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=True)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(64, activation='relu', kernel_regularizer='l2')(x)
combined_output = tf.keras.layers.Dense(2, activation='linear', name='combined_output')(x)

# Define the model
model = tf.keras.Model(inputs, combined_output)

94765736/94765736 [==============================] - 2s 0us/step


In [ ]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=tf.keras.losses.MeanSquaredError())

In [ ]:
# Define ModelCheckpoint callback
checkpoint_filepath = '/content/drive/My Drive/MLiS_Car/ResNet_50e'
model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

# Training loop
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,
    callbacks=[model_checkpoint]  # Include the ModelCheckpoint callback
)

Epoch 1/50
301/301 [==============================] - ETA: 0s - loss: 1.2476
Epoch 1: val_loss improved from inf to 1.04381, saving model to /content/drive/My Drive/MLiS_Car/ResNet_50e
301/301 [==============================] - 114s 249ms/step - loss: 1.2476 - val_loss: 1.0438
Epoch 2/50
301/301 [==============================] - ETA: 0s - loss: 0.9027
Epoch 2: val_loss improved from 1.04381 to 0.76231, saving model to /content/drive/My Drive/MLiS_Car/ResNet_50e
301/301 [==============================] - 72s 240ms/step - loss: 0.9027 - val_loss: 0.7623
Epoch 3/50
301/301 [==============================] - ETA: 0s - loss: 0.6424
Epoch 3: val_loss improved from 0.76231 to 0.53013, saving model to /content/drive/My Drive/MLiS_Car/ResNet_50e
301/301 [==============================] - 69s 229ms/step - loss: 0.6424 - val_loss: 0.5301
Epoch 4/50
301/301 [==============================] - ETA: 0s - loss: 0.4353
Epoch 4: val_loss improved from 0.53013 to 0.35322, saving model to /content/drive/

**TFLite Model Conversion**

In [ ]:
# best_model = tf.keras.models.load_model(checkpoint_filepath)

best_model = tf.keras.models.load_model('/content/drive/My Drive/MLiS_Car/ResNet_50e')

In [ ]:
# Evaluate on the test set
best_model.evaluate(test_dataset)

print('Finished Training')

64/64 [==============================] - 10s 126ms/step - loss: 0.0086
Finished Training


In [ ]:
# Set up the TFLite converter
converter = tf.lite.TFLiteConverter.from_keras_model(best_model)

# Enable specific conversion options to improve compatibility
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # Allow TensorFlow ops.
]

# Convert the model
tflite_model = converter.convert()

path_to_save = '/content/drive/My Drive/MLiS_Car/best_res.tflite'
with open(path_to_save, 'wb') as f:
    f.write(tflite_model)